# ============================================================
## Семинар 6
### Классификация изображений на примере фотографий котов и собак.
## DLSchool
# ============================================================

**Мотивировка:** В 2010-х годах компанией $Google$ в качестве подтверждения того, что пользователь -- не робот, предлагалось из некоторого набора картинок выборать всех собак или всех котов. Если какой-нибудь питомец особенно понравился, можно было пройти по ссылке и забрать его из приюта. 
Но прогресс не стоял на месте, нейронные сети научились проходить "тест на человека". Такую сеть мы сейчас и рассмотрим.

In [19]:
import sys
import os
import os.path
import time
import glob
import random
import collections
import shutil
import csv
import numpy as np

import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

from PIL import Image

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data'
%matplotlib inline

In [20]:
# пути по директориям
data_path = DATA_HOME_DIR + '/' 
split_train_path = data_path + '/train/'
full_train_path = data_path + '/train_full/'
valid_path = data_path + '/valid/'
test_path = DATA_HOME_DIR + '/test/test/'
saved_model_path = ROOT_DIR + '/models/'
submission_path = ROOT_DIR + '/submissions/'

# данные
batch_size = 8
nb_split_train_samples = 23000
nb_full_train_samples = 25000
nb_valid_samples = 2000
nb_test_samples = 12500

# параметры модели
nb_runs = 1
nb_aug = 3
epochs = 35
lr = 1e-4
clip = 0.001
archs = ["resnet152"]

model_names = sorted(name for name in models.__dict__ if name.islower() and not name.startswith("__"))
best_prec1 = 0

**Вопрос:** что лежит внутри model_names?

In [21]:
model_names

['alexnet',
 'densenet',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'inception',
 'inception_v3',
 'resnet',
 'resnet101',
 'resnet152',
 'resnet18',
 'resnet34',
 'resnet50',
 'squeezenet',
 'squeezenet1_0',
 'squeezenet1_1',
 'vgg',
 'vgg11',
 'vgg11_bn',
 'vgg13',
 'vgg13_bn',
 'vgg16',
 'vgg16_bn',
 'vgg19',
 'vgg19_bn']

### Некоторые вспомогательные функции 
**Вопрос: что конкретно делает каждая из функций? Можно ли как-то заоптимизировать?

In [22]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()
    end = time.time()
    
    # переключение на обучение
    model.train()
    
    for i, (images, target) in enumerate(train_loader):
        # время загрузки
        data_time.update(time.time() - end)

        target = target.cuda(async=True)
        image_var = torch.autograd.Variable(images)
        label_var = torch.autograd.Variable(target)

        # вычисление y_pred
        y_pred = model(image_var)
        loss = criterion(y_pred, label_var)

        # вычисляем точность и ошибкую 
        prec1, prec1 = accuracy(y_pred.data, target, topk=(1, 1))
        losses.update(loss.data[0], images.size(0))
        acc.update(prec1[0], images.size(0))

        # Градиент
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        batch_time.update(time.time() - end)
        end = time.time()

In [23]:
def validate(val_loader, model, criterion, epoch):
    batch_time = AverageMeter()
    losses = AverageMeter()
    acc = AverageMeter()

    # оцениваем модель
    model.eval()

    end = time.time()
    for i, (images, labels) in enumerate(val_loader):
        labels = labels.cuda(async=True)
        image_var = torch.autograd.Variable(images, volatile=True)
        label_var = torch.autograd.Variable(labels, volatile=True)

        # снова вычисляем y_pred
        y_pred = model(image_var)
        loss = criterion(y_pred, label_var)

        # снова точность и ошибка
        prec1, temp_var = accuracy(y_pred.data, labels, topk=(1, 1))
        losses.update(loss.data[0], images.size(0))
        acc.update(prec1[0], images.size(0))

        batch_time.update(time.time() - end)
        end = time.time()

    return acc.avg

In [24]:
def test(test_loader, model):
    csv_map = collections.defaultdict(float)
    
    model.eval()
    
    for aug in range(nb_aug):
        print("   * Предсказание увеличения {}".format(aug + 1))
        
        for i, (images, filepath) in enumerate(test_loader):
            # расширение как идентификатор мапа
            filepath = os.path.splitext(os.path.basename(filepath[0]))[0]
            filepath = int(filepath)

            image_var = torch.autograd.Variable(images, volatile=True)
            y_pred = model(image_var)
            # получение индекса с максимальным значением вероятности
            smax = nn.Softmax()
            smax_out = smax(y_pred)[0]
            cat_prob = smax_out.data[0]
            dog_prob = smax_out.data[1]
            prob = dog_prob
            if cat_prob > dog_prob:
                prob = 1 - cat_prob
            prob = np.around(prob, decimals=4)
            prob = np.clip(prob, clip, 1-clip)
            csv_map[filepath] += (prob / nb_aug)

    sub_fn = submission_path + '{0}epoch_{1}clip_{2}runs'.format(epochs, clip, nb_runs)
    
    for arch in archs:
        sub_fn += "_{}".format(arch)
        
    print("Writing Predictions to CSV...")
    with open(sub_fn + '.csv', 'w') as csvfile:
        fieldnames = ['id', 'label']
        csv_w = csv.writer(csvfile)
        csv_w.writerow(('id', 'label'))
        for row in sorted(csv_map.items()):
            csv_w.writerow(row)
    print("Done.")

### Напишем функцию, которая вычисляет и сохраняет среднее и текущее значение:

In [25]:
class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

Теперь реализуем функцию, которая устанавливает скорость обучения для начального LR, 
разлагающегося на 10 каждые 30 периодов

In [26]:
def adjust_learning_rate(optimizer, epoch):
    global lr
    lr = lr * (0.1**(epoch // 30))
    for param_group in optimizer.state_dict()['param_groups']:
        param_group['lr'] = lr

### Вычислим точность нашего алгоритма:

In [27]:
def accuracy(y_pred, y_actual, topk=(1, )):
    maxk = max(topk)
    batch_size = y_actual.size(0)

    _, pred = y_pred.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(y_actual.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))

    return res

In [28]:
class TestImageFolder(data.Dataset):
    def __init__(self, root, transform=None):
        images = []
        for filename in sorted(glob.glob(test_path + "*.jpg")):
            images.append('{}'.format(filename))

        self.root = root
        self.imgs = images
        self.transform = transform

    def __getitem__(self, index):
        filename = self.imgs[index]
        img = Image.open(os.path.join(self.root, filename))
        if self.transform is not None:
            img = self.transform(img)
        return img, filename

    def __len__(self):
        return len(self.imgs)

In [29]:
def shear(img):
    width, height = img.size
    m = random.uniform(-0.05, 0.05)
    xshift = abs(m) * width
    new_width = width + int(round(xshift))
    img = img.transform((new_width, height), Image.AFFINE,
                        (1, m, -xshift if m > 0 else 0, 0, 1, 0),
                        Image.BICUBIC)
    return img

### Обучение

In [36]:
def training(mode="train", resume=False):
    
    global best_prec1
    
    for arch in archs:
    # создали модель
        print("=> Starting {0} on '{1}' model".format(mode, arch))
        model = models. __ dict __ [arch] (prerained = True)
        # Не обновляем извлеченные функции, 
        #не связанные с классификатором, в предобученных сетях для параметра 
        param.parameters ():
             param.requires_grad = False
         # Заменим последний полностью подключенный уровень
         # Параметры вновь построенных модулей имеют require_grad = True по умолчанию
         #количество классов в этом случае -- resnet 101 -- это 2048 с двумя классами (кошки и собаки)
         model.fc = nn.Linear (2048, 2)

        if arch.startswith('alexnet') or arch.startswith('vgg'):
            model.features = torch.nn.DataParallel(model.features)
            model.cuda()
        else:
            model = torch.nn.DataParallel(model).cuda()
            
        cudnn.benchmark = True

        # Загрузка данных
        traindir = split_train_path
        valdir = valid_path
        testdir = test_path

        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

        train_loader = data.DataLoader(
            datasets.ImageFolder(traindir,
                                 transforms.Compose([
                                     transforms.RandomSizedCrop(224),
                                     transforms.RandomHorizontalFlip(),
                                     transforms.ToTensor(),
                                     normalize,
                                 ])),
            batch_size=batch_size,
            shuffle=True,
            num_workers=4,
            pin_memory=True)

        val_loader = data.DataLoader(
            datasets.ImageFolder(valdir,
                                 transforms.Compose([
                                     transforms.Scale(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     normalize,
                                 ])),
            batch_size=batch_size,
            shuffle=True,
            num_workers=4,
            pin_memory=True)

        test_loader = data.DataLoader(
            TestImageFolder(testdir,
                            transforms.Compose([
                                transforms.Scale(256),
                                transforms.CenterCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                normalize,
                            ])),
            batch_size=1,
            shuffle=False,
            num_workers=1,
            pin_memory=False)
        
        
        if mode == "test":
            test(test_loader, model)
            return
        
        # опеределение функции потерь
        criterion = nn.CrossEntropyLoss().cuda()
        
        if mode == "validate":
            validate(val_loader, model, criterion, 0)
            return

        optimizer = optim.Adam(model.module.fc.parameters(), lr, weight_decay=1e-4)

        for epoch in range(epochs):
            adjust_learning_rate(optimizer, epoch)

            # обучение одного периода
            train(train_loader, model, criterion, optimizer, epoch)

            # оценка
            prec1 = validate(val_loader, model, criterion, epoch)


SyntaxError: invalid character in identifier (<ipython-input-36-bd9e599e12c2>, line 8)

### Run Train

In [13]:
training(mode="train")

=> Starting train on 'resnet152' model


Downloading: "https://s3.amazonaws.com/pytorch/models/resnet152-b121ed2d.pth" to /home/robert/.torch/models/resnet152-b121ed2d.pth
100%|██████████| 241530880/241530880 [02:04<00:00, 1936481.28it/s]


   * EPOCH 0 | Accuracy: 98.800 | Loss: 0.048
   * EPOCH 1 | Accuracy: 99.000 | Loss: 0.041
   * EPOCH 2 | Accuracy: 98.850 | Loss: 0.038
   * EPOCH 3 | Accuracy: 98.850 | Loss: 0.039
   * EPOCH 4 | Accuracy: 98.900 | Loss: 0.035
   * EPOCH 5 | Accuracy: 98.950 | Loss: 0.036
   * EPOCH 6 | Accuracy: 98.850 | Loss: 0.037
   * EPOCH 7 | Accuracy: 98.950 | Loss: 0.036
   * EPOCH 8 | Accuracy: 99.000 | Loss: 0.034
   * EPOCH 9 | Accuracy: 98.900 | Loss: 0.036
   * EPOCH 10 | Accuracy: 98.900 | Loss: 0.036
   * EPOCH 11 | Accuracy: 98.800 | Loss: 0.039
   * EPOCH 12 | Accuracy: 99.000 | Loss: 0.034
   * EPOCH 13 | Accuracy: 98.800 | Loss: 0.041
   * EPOCH 14 | Accuracy: 98.900 | Loss: 0.035
   * EPOCH 15 | Accuracy: 99.000 | Loss: 0.037
   * EPOCH 16 | Accuracy: 98.900 | Loss: 0.036
   * EPOCH 17 | Accuracy: 99.150 | Loss: 0.033
   * EPOCH 18 | Accuracy: 98.950 | Loss: 0.033
   * EPOCH 19 | Accuracy: 98.900 | Loss: 0.037
   * EPOCH 20 | Accuracy: 98.900 | Loss: 0.036
   * EPOCH 21 | Accurac

In [14]:
main(mode="validate", resume='model_best.pth.tar')

=> Starting validate on 'resnet152' model
=> Loading checkpoint 'model_best.pth.tar'
=> Loaded checkpoint (epoch 18)
   * EPOCH 0 | Accuracy: 99.150 | Loss: 0.033


### Тестирование

In [15]:
main(mode="test", resume='model_best.pth.tar')

=> Starting test on 'resnet152' model
=> Loading checkpoint 'model_best.pth.tar'
=> Loaded checkpoint (epoch 18)
   * Predicting on test augmentation 1
   * Predicting on test augmentation 2
   * Predicting on test augmentation 3
Writing Predictions to CSV...
Done.
